In [1]:
# Copyright (c) Facebook, Inc. and its affiliates. All Rights Reserved
# -*- coding: utf-8 -*-
"""
Demo code

@author: Denis Tome'

"""
from torch.utils.data import DataLoader
import torch
from torchvision import transforms
from base import SetType

from mocap import Mocap
from utils import config, ConsoleLogger
from utils import evaluate, io
from model import TempModel
from tqdm import tqdm

LOGGER = ConsoleLogger("Main")
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

def main():
	"""Main"""

	LOGGER.info('Starting demo...')

	# ------------------- Data loader -------------------


	# let's load data from validation set as example
	data = Mocap(
		config.dataset.test,
		SetType.VAL,
		)
	data_loader = DataLoader(
		data,
		batch_size=config.data_loader.batch_size,
		shuffle=config.data_loader.shuffle)

	# ------------------- Evaluation -------------------

	eval_body = evaluate.EvalBody()
	eval_upper = evaluate.EvalUpperBody()
	eval_lower = evaluate.EvalLowerBody()

	# ------------------- Read dataset frames -------------------

	model = TempModel().to(device)
	model = torch.nn.DataParallel(model,device_ids=[0,1])
	model.eval()

	for it, test_batch in enumerate(tqdm(data_loader)):
		img = test_batch['image']
		p2d = test_batch['fisheye_joints_2d']
		p3d = test_batch['joints_3d_cam']
		action = test_batch['action']
		input_img = {'image':img}
		# LOGGER.info('Iteration: {}'.format(it))
		# LOGGER.info('Images: {}'.format(img.shape))
		# LOGGER.info('p2ds: {}'.format(p2d.shape))
		# LOGGER.info('p3ds: {}'.format(p3d.shape))
		# LOGGER.info('Actions: {}'.format(action))

		# -----------------------------------------------------------
		pred_output = model(input_img,0,False)
		# -----------------------------------------------------------

		# TODO: replace p3d_hat with model preditions
		p3d_hat = pred_output['regressor_dict']['pred_joint']

		# Evaluate results using different evaluation metrices
		y_output = p3d_hat.data.cpu().numpy() * 1000
		y_target = p3d.data.cpu().numpy() * 1000

		eval_body.eval(y_output, y_target, action)
		eval_upper.eval(y_output, y_target, action)
		eval_lower.eval(y_output, y_target, action)
		if (it+1)%200==0:
			print({'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()})
		# TODO: remove break
		

	# ------------------- Save results -------------------

	LOGGER.info('Saving evaluation results...')
	res = {'FullBody': eval_body.get_results(),
		   'UpperBody': eval_upper.get_results(),
		   'LowerBody': eval_lower.get_results()}

	io.write_json(config.eval.output_file, res)

	LOGGER.info('Done.')

In [3]:
main()

03:10:20 [INFO] Main: Starting demo...
  0%|          | 0/1807 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:329: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(
 11%|█         | 200/1807 [15:23<2:01:43,  4.54s/it]

{'FullBody': {'All': 142.79815673828125, 'UpperStretching': 160.58416748046875, 'Gaming': 84.51429748535156, 'Talking': 87.72078704833984, 'Greeting': 77.27350616455078, 'LowerStretching': 159.92605590820312, 'Gesticuling': 81.41439819335938, 'Walking': 139.30935668945312, 'Reacting': 89.84170532226562, 'Patting': 94.32389831542969}, 'UpperBody': {'All': 89.7162857055664, 'UpperStretching': 102.25479125976562, 'Gaming': 59.3114128112793, 'Talking': 63.39605712890625, 'Greeting': 62.93657302856445, 'LowerStretching': 83.70323944091797, 'Gesticuling': 68.59809875488281, 'Walking': 84.25880432128906, 'Reacting': 69.39208221435547, 'Patting': 58.62029266357422}, 'LowerBody': {'All': 195.88003540039062, 'UpperStretching': 218.91355895996094, 'Gaming': 109.71719360351562, 'Talking': 112.04552459716797, 'Greeting': 91.61043548583984, 'LowerStretching': 236.14891052246094, 'Gesticuling': 94.23068237304688, 'Walking': 194.35989379882812, 'Reacting': 110.29132080078125, 'Patting': 130.0274963378

 11%|█▏        | 204/1807 [15:42<2:06:12,  4.72s/it]